In [15]:
suppressMessages(library(dplyr))
suppressMessages(library(reshape2))
suppressMessages(library(ggplot2))
suppressMessages(library(ggtern))
library(cowplot)
library(tidyr, warn.conflicts = FALSE)

### Prokaryotes

In [25]:
df <- read.table(file = "~/2021-marine-osmolytes/genome-searching/predict_synthesis/2021-03-predicted-bacterial-osmolyte-synthesis.tsv", sep = '\t', header = F)
dim(df)
#head(df)
osmo <- df[1,2:dim(df)[2]]
type <- df[2,2:dim(df)[2]]
# simplify the df
df <- df[-c(1:2),]
rownames(df) <- df[,1]
df <- df[,-1]

# create taxa table
marref <- read.delim("~/2021-marine-osmolytes/genome-searching/MarRef_DBs/MarRef_v5.txt")
dim(marref)
# removing the marref with NA's in accession value
idx <- which(is.na(marref$assembly_accession_refseq))
marref <- marref[-idx,]
# replace rownames with accession value
rownames(marref) <- marref$assembly_accession_refseq
dim(marref)
# rearrange so that rownames are equal to the synthesis tsv file
marref <- marref[rownames(df),]
all(rownames(df) == marref$assembly_accession_refseq)

# create synthesis df
syn <- data.frame(sapply(df[,which(type == "SYNTHESIS")], as.numeric))
rownames(syn) <- rownames(df)
colnames(syn) <- osmo[which(type == "SYNTHESIS")]
#head(syn)
# create breakdown df
breakd <- data.frame(sapply(df[,which(type == "BREAKDOWN")], as.numeric))
rownames(breakd) <- rownames(df)
colnames(breakd) <- osmo[which(type == "BREAKDOWN")]
#head(breakd)
dim(syn) == dim(breakd)
# create both df
both <- as.matrix(syn) + as.matrix(breakd)
#head(both)
# remove duplicates from syn/breakd, place 0's if both ==2
for(i in 1:dim(syn)[1]){
    idx <- which(both[i,] == 1)
    both[i,idx] <- 0 #replace 1's with 0
    idx <- which(both[i,] == 2) # if both =1
    syn[i,idx] <- 0
    breakd[i,idx] <- 0
    both[i,idx] <- 1
}

[1] 899  49

[1] 970 126

[1] 908 126

[1] TRUE

[1] TRUE TRUE

In [26]:
## Adding a custom classification 
marref$custom <- marref$phylum
idx <- marref$custom == "Proteobacteria" # Split proteobacteria into delta, alpha, beta, gamma, epsilon, zeta, oligoflex
marref$custom[idx] <- marref$class[idx]

idx <- marref$kingdom == "Archaea" # combine Eury and Crenarchaeota
marref$custom[idx] <- "Archaea"

idx <- marref$custom == "Actinobacteria (Phylum)" # drop the extra label for Actinos
marref$custom[idx] <- "Actinobacteria"

unique(marref$custom)

[1] "Deltaproteobacteria"            "Alphaproteobacteria"           
 [3] "Betaproteobacteria"             "Cyanobacteria"                 
 [5] "Gammaproteobacteria"            "Bacteroidetes"                 
 [7] "Archaea"                        "Actinobacteria"                
 [9] "Firmicutes"                     "Thermotogae (Phylum)"          
[11] "Aquificae"                      "Calditrichaeota"               
[13] "Chlorobi"                       "Epsilonproteobacteria"         
[15] "Deferribacteres (Phylum)"       "Unclassified"                  
[17] "Planctomycetes"                 "Verrucomicrobia"               
[19] "Tenericutes"                    "Chloroflexi"                   
[21] "Spirochaetes"                   "Zetaproteobacteria"            
[23] "Deinococcus-Thermus"            "Thermodesulfobacteria (Phylum)"
[25] "Aquificae (Phylum)"             "Synergistetes"                 
[27] "Oligoflexia"                    "Fusobacteria"

In [27]:
df.sums <- data.frame(synthesis = colSums(syn), breakdown = colSums(breakd), both = colSums(both))
df.sums$total <- rowSums(df.sums)
df.sums$all <- dim(marref)[1]
df.sums$synper <- (df.sums$synthesis/df.sums$total)*100
df.sums$breakper <- (df.sums$breakdown/df.sums$total)*100
df.sums$bothper <- (df.sums$both/df.sums$total)*100
df.sums <- df.sums[-which(rownames(df.sums) == "DMSP" | rownames(df.sums) == "Hydroxyectoine"),]
df.sums

,synthesis,breakdown,both,total,all,synper,breakper,bothper
,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
Ectoine,39,47,90,176,897,22.1590909,26.7045455,51.13636
GABA,55,225,52,332,897,16.5662651,67.7710843,15.66265
Glutamate,0,0,897,897,897,0.0000000,0.0000000,100.00000
Glutamine,0,2,895,897,897,0.0000000,0.2229654,99.77703
Glycerol,26,282,409,717,897,3.6262204,39.3305439,57.04324
Glycine betaine,316,4,0,320,897,98.7500000,1.2500000,0.00000
Mannitol,8,79,214,301,897,2.6578073,26.2458472,71.09635
Proline,0,59,838,897,897,0.0000000,6.5774805,93.42252
Sarcosine,1,177,86,264,897,0.3787879,67.0454545,32.57576


In [28]:
df.sumsbac <- df.sums

In [29]:
df.sums$cmpd <- rownames(df.sums)
# we create the ternary plot
cmpd_colors <-c('#d4b9da','#c994c7','#df65b0','#e7298a','#ce1256','#980043','#67001f',
                '#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529')
names(cmpd_colors) <- df.sums$cmpd
min(df.sums$total)
max(df.sums$total)
p2 <- ggtern(df.sums, aes(x = both, y = synthesis, z = breakdown, color = cmpd, size = total))+
                geom_mask()+
                geom_point()+scale_color_manual(values = cmpd_colors)+
                scale_size(limits = c(38,897), range = c(0.5,15), breaks = c(40,400,897))+
                theme_rgbw()
pdf("ternaryprok.pdf",width = 12, height = 8)
p2
dev.off()

[1] 40

[1] 897

png 
  2

### Eukaryotes

In [30]:
df <- read.table(file = "~/2021-marine-osmolytes/genome-searching/predict_synthesis/2021-03-predicted-mmetsp-osmolyte-synthesis.tsv", sep = '\t', header = F)
dim(df)
#head(df)
osmo <- df[1,2:dim(df)[2]]
type <- df[2,2:dim(df)[2]]
# simplify the df
df <- df[-c(1:2),]
rownames(df) <- df[,1]
df <- df[,-1]

# create taxa table
#marref <- read.delim("~/2021-marine-osmolytes/genome-searching/MarRef_DBs/MarRef_v5.txt")
#dim(marref)
# removing the marref with NA's in accession value
#idx <- which(is.na(marref$assembly_accession_refseq))
#marref <- marref[-idx,]
# replace rownames with accession value
#rownames(marref) <- marref$assembly_accession_refseq
#dim(marref)
# rearrange so that rownames are equal to the synthesis tsv file
#marref <- marref[rownames(df),]
#all(rownames(df) == marref$assembly_accession_refseq)

# create synthesis df
syn <- data.frame(sapply(df[,which(type == "SYNTHESIS")], as.numeric))
rownames(syn) <- rownames(df)
colnames(syn) <- osmo[which(type == "SYNTHESIS")]
#head(syn)
# create breakdown df
breakd <- data.frame(sapply(df[,which(type == "BREAKDOWN")], as.numeric))
rownames(breakd) <- rownames(df)
colnames(breakd) <- osmo[which(type == "BREAKDOWN")]
#head(breakd)
dim(syn) == dim(breakd)
# create both df
both <- as.matrix(syn) + as.matrix(breakd)
#head(both)
# remove duplicates from syn/breakd, place 0's if both ==2
for(i in 1:dim(syn)[1]){
    idx <- which(both[i,] == 1)
    both[i,idx] <- 0 #replace 1's with 0
    idx <- which(both[i,] == 2) # if both =1
    syn[i,idx] <- 0
    breakd[i,idx] <- 0
    both[i,idx] <- 1
}
#head(both)

[1] 654  49

[1] TRUE TRUE

In [31]:
df.sums <- data.frame(synthesis = colSums(syn), breakdown = colSums(breakd), both = colSums(both))
df.sums$total <- rowSums(df.sums)
df.sums$all <- dim(syn)[1]
df.sums$synper <- (df.sums$synthesis/df.sums$total)*100
df.sums$breakper <- (df.sums$breakdown/df.sums$total)*100
df.sums$bothper <- (df.sums$both/df.sums$total)*100
df.sums <- df.sums[-which(rownames(df.sums) == "DMSP" | rownames(df.sums) == "Hydroxyectoine"),]
df.sums

,synthesis,breakdown,both,total,all,synper,breakper,bothper
,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
Ectoine,1,37,0,38,652,2.6315789,97.3684211,0.000000
GABA,78,125,62,265,652,29.4339623,47.1698113,23.396226
Glutamate,2,2,647,651,652,0.3072197,0.3072197,99.385561
Glutamine,3,4,644,651,652,0.4608295,0.6144393,98.924731
Glycerol,18,49,511,578,652,3.1141869,8.4775087,88.408304
Glycine betaine,51,28,2,81,652,62.9629630,34.5679012,2.469136
Mannitol,44,0,37,81,652,54.3209877,0.0000000,45.679012
Proline,0,44,597,641,652,0.0000000,6.8642746,93.135725
Sarcosine,5,164,17,186,652,2.6881720,88.1720430,9.139785


In [32]:
df.sumseuk <- df.sums

In [33]:
df.sums$cmpd <- rownames(df.sums)
# we create the ternary plot
cmpd_colors <-c('#d4b9da','#c994c7','#df65b0','#e7298a','#ce1256','#980043','#67001f',
                '#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529')
names(cmpd_colors) <- df.sums$cmpd
min(df.sums$total)
max(df.sums$total)
p2 <- ggtern(df.sums, aes(x = both, y = synthesis, z = breakdown, color = cmpd, size = total))+
                geom_mask()+
                geom_point()+scale_color_manual(values = cmpd_colors)+
                scale_size(limits = c(38,652), range = c(0.5,15), breaks = c(40,300,652))+
                theme_rgbw()
pdf("ternaryeuk.pdf",width = 12, height = 8)
p2
dev.off()

[1] 38

[1] 651

png 
  2

## Comparing totals vs. pathway and step numbers

In [34]:
## Pull this dataframe from Harriet's global analysis.ipynb used for assigning presence/absence of completed pathways
osmos <- read.csv("~/2021-marine-osmolytes/genome-searching/predict_synthesis/osmo_machineread_subsetdf.csv") %>% 
            group_by(COMBINED_ORTHO) %>% 
            dplyr::distinct(COMBINED_ORTHO, .keep_all = TRUE)

osmos$BROAD[which(osmos$BROAD == "Breakdown")] <- "BREAKDOWN"

paths <- osmos %>%
  filter(COMPOUND_NAME != "DMSP" & COMPOUND_NAME != "Hydroxyectoine") %>%
  group_by(COMPOUND_NAME,BROAD) %>%
  summarise(tot = n(), min = min(max_step), max = max(max_step)) %>% 
  pivot_wider(names_from = BROAD, values_from = c(tot,min,max))
colnames(paths)[1] <- "cmpd"

`summarise()` regrouping output by 'COMPOUND_NAME' (override with `.groups` argument)



In [35]:
df.sumsbac$cmpd <- rownames(df.sumsbac)
df.sumseuk$cmpd <- rownames(df.sumseuk)
toplot.bac <- merge(df.sumsbac, paths, by = "cmpd")
toplot.bac$synthesis <- toplot.bac$synthesis + toplot.bac$both
toplot.bac$breakdown <- toplot.bac$breakdown + toplot.bac$both
toplot.euk <- merge(df.sumseuk, paths, by = "cmpd")
toplot.euk$synthesis <- toplot.euk$synthesis + toplot.euk$both
toplot.euk$breakdown <- toplot.euk$breakdown + toplot.euk$both

b <- ggplot()+geom_point(data=toplot.bac, aes(x = tot_BREAKDOWN, y = breakdown, shape = "prok", color = cmpd, size = 2))+
                geom_point(data=toplot.euk, aes(x = tot_BREAKDOWN, y = breakdown, shape = "euk", color = cmpd, size = 2))+
                xlab("no. of Pathways")+ylab("no. of Genomes")+ggtitle("Breakdown")+
                theme(text = element_text(size=14))
s <- ggplot()+geom_point(data=toplot.bac, aes(x = tot_SYNTHESIS, y = synthesis, shape = "prok", color = cmpd, size = 2))+
                geom_point(data=toplot.euk, aes(x = tot_SYNTHESIS, y = synthesis, shape = "euk", color = cmpd, size = 2, size = 2))+
                xlab("no. of Pathways")+ylab("no. of Genomes")+ggtitle("Synthesis")+
                theme(text = element_text(size=14))
options(repr.plot.width=16, repr.plot.height=8)
pdf("pathwaycompare.pdf",width = 12, height = 8)
plot_grid(b,s)
dev.off()

Warning message:
“The plyr::rename operation has created duplicates for the following name(s): (`size`)”


png 
  2

In [36]:
b <- ggplot()+geom_point(data=toplot.bac, aes(x = max_BREAKDOWN, y = breakdown, shape = "prok", color = cmpd, size = 2))+
                geom_point(data=toplot.euk, aes(x = max_BREAKDOWN, y = breakdown, shape = "euk", color = cmpd, size = 2))+
                xlab("max no. of Steps")+ylab("no. of Genomes")+ggtitle("Breakdown")+
                theme(text = element_text(size=14))
s <- ggplot()+geom_point(data=toplot.bac, aes(x = max_SYNTHESIS, y = synthesis, shape = "prok", color = cmpd, size = 2))+
                geom_point(data=toplot.euk, aes(x = max_SYNTHESIS, y = synthesis, shape = "euk", color = cmpd, size = 2))+
                xlab("max no. of Steps")+ylab("no. of Genomes")+ggtitle("Synthesis")+
                theme(text = element_text(size=14))
options(repr.plot.width=16, repr.plot.height=8)
pdf("stepcompare.pdf",width = 12, height = 8)
plot_grid(b,s)
dev.off()

png 
  2

In [37]:
toplot.bac

cmpd,synthesis,breakdown,both,total,all,synper,breakper,bothper,tot_BREAKDOWN,tot_SYNTHESIS,tot_TRANSPORT,min_BREAKDOWN,min_SYNTHESIS,min_TRANSPORT,max_BREAKDOWN,max_SYNTHESIS,max_TRANSPORT
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Ectoine,129,137,90,176,897,22.1590909,26.7045455,51.13636,2,2,NA,1,4,NA,3,5,NA
GABA,107,277,52,332,897,16.5662651,67.7710843,15.66265,4,11,NA,1,1,NA,2,4,NA
Glutamate,897,897,897,897,897,0.0000000,0.0000000,100.00000,61,78,NA,1,1,NA,5,3,NA
Glutamine,895,897,895,897,897,0.0000000,0.2229654,99.77703,30,2,NA,1,1,NA,1,1,NA
Glycerol,435,691,409,717,897,3.6262204,39.3305439,57.04324,7,9,3,1,1,1,3,4,5
Glycine betaine,316,4,0,320,897,98.7500000,1.2500000,0.00000,1,4,2,3,1,3,3,2,3
Mannitol,222,293,214,301,897,2.6578073,26.2458472,71.09635,2,1,1,1,2,4,2,2,4
Proline,838,897,838,897,897,0.0000000,6.5774805,93.42252,12,6,NA,1,1,NA,1,2,NA
Sarcosine,87,263,86,264,897,0.3787879,67.0454545,32.57576,2,7,NA,1,1,NA,1,4,NA
